In [2]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

from torch.optim import Adam
from torch.utils.data import DataLoader,random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image

from livelossplot import PlotLosses

In [7]:
im = Image.open("data/train/Pause/20191027_023010_098.jpg")
im.show()

In [2]:
IMAGE_X = 50
IMAGE_Y = 50

TRAIN_DATA_PATH = 'data/train/'

VALIDATION_SPLIT_SIZE = 0.1

BATCH_SIZE = 4
NUM_WORKERS = 8
SHUFFLE = True

LR = 0.01
EPOCH = 10

DROPOUT = 0.1

def add_channels(img):
    openCVim = np.array(img)
#     print(openCVim.shape)

    img_gray = cv2.cvtColor(openCVim, cv2.COLOR_BGR2GRAY)
    img_gray = img_gray[:,:,np.newaxis]
#     print(img_gray.shape)
    
    img_combined = np.concatenate((openCVim, img_gray), axis=2)
#     print(img_combined.shape)
    PILim = Image.fromarray(img_combined)

    return PILim

TRANSFORM = transforms.Compose(
[
#     transforms.Lambda(add_channels),
transforms.Resize((IMAGE_X,IMAGE_Y)),
 transforms.ToTensor(),
#  transforms.Normalize((0.5, 0.5, 0.5, 0.5), (0.5, 0.5, 0.5, 0.5))])
 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
def load_dataset_from_folder():
    all_data = ImageFolder(
        root = TRAIN_DATA_PATH,
        transform = TRANSFORM
    )
    
    classes = all_data.classes
    print(all_data.class_to_idx)
    validation_size = int(VALIDATION_SPLIT_SIZE * len(all_data))
    train_size = len(all_data) - validation_size
    train_dataset, test_dataset = torch.utils.data.random_split(all_data, [train_size, validation_size])

    training_data_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=SHUFFLE
    )
    
    validation_dataset_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=SHUFFLE
    )
        
    return training_data_loader,validation_dataset_loader,classes

class Net(nn.Module):
    fin_x = 0
    fin_y = 0
    k_size = 3
    stride = 2
    pool_size = 2
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(4, 32, self.k_size, self.stride),
            nn.Dropout2d(DROPOUT),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size, self.pool_size))

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, self.k_size, self.stride),
            nn.Dropout2d(DROPOUT),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size, self.pool_size))
        
        self.drop_out = nn.Dropout(DROPOUT)

        
        tmpx = int((int((IMAGE_X-self.k_size)/self.stride)+1)/self.pool_size)
        tmpy = int((int((IMAGE_Y-self.k_size)/self.stride)+1)/self.pool_size)
        
        self.fin_x = int((int((tmpx-self.k_size)/self.stride)+1)/self.pool_size)
        self.fin_y = int((int((tmpy-self.k_size)/self.stride)+1)/self.pool_size)
        
        self.fc1 = nn.Sequential(
            nn.Linear(64 * self.fin_x * self.fin_y, 2048),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(2048, 4),
            nn.ReLU()
        )
        
        self.sof = nn.LogSoftmax()

    def forward(self, inp):
        out = self.layer1(inp)
        out = self.layer2(out)
        out = out.view(-1, 64 * self.fin_x * self.fin_y)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sof(out)
        return out


In [ ]:
model = Net()

# Defining the optimizer
optimizer = Adam(model.parameters(), lr=LR)

# Defining the loss function
criterion = nn.CrossEntropyLoss()

# Checking if GPU is available
if torch.cuda.is_available():
    print('Found Cuda')
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
train_generator,test_generator,classes = load_dataset_from_folder()
print(classes)
# for x,y in train_generator:
#     continue

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def get_rgb_img(tensor_batch):
    modif = tensor_batch[:,:3,:,:]
    return modif

def get_grayscale(tensor_batch):
    modif = tensor_batch[:,3:4,:,:]
    return modif

# get some random training images
dataiter = iter(train_generator)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(get_rgb_img(images)))
# imshow(torchvision.utils.make_grid(get_grayscale(images)))

# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
liveloss = PlotLosses()

for epoch in range(EPOCH):
    logs = {}

    running_loss = 0.0
    running_acc = 0.0
    for step,(x,y) in enumerate(train_generator):
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        running_acc += torch.sum(preds == y)
        
    epoch_acc = running_acc.float() / len(train_generator.dataset)

#     print('[%d] Training loss: %.3f' %(epoch + 1, running_loss))

#     logs['log loss'] = running_loss
    logs['accuracy'] = epoch_acc
    
    validation_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for step,(x,y) in enumerate(test_generator):
            outputs = model(x)
            loss = criterion(outputs, y)
            _, preds = torch.max(outputs, 1)
            validation_loss += loss.item()
            val_acc += torch.sum(preds == y)
#     print('[%d] Validation loss: %.3f' %(epoch + 1, validation_loss))
    
    val_fin_acc = val_acc.float() / len(test_generator.dataset)
#     logs['val_log loss'] = validation_loss
    logs['val_accuracy'] = val_fin_acc
    liveloss.update(logs)
    liveloss.draw()